# **Logistic Regression**

We have now recapped on inferential statistics, Analysis of Variance and Linear regression. But students always want to know how do I model categorical variables, or how do I classify a piece of data. Logistic regression is one of the simplest classifiers and we will give a quick introduction to it in this section. However, logistic regression comes from the Generalised Linear model family of models and we will examine it in much more detail in the later.

<br/>

The objective of Logistic Regression is to model binary outcomes. In linear regression we modelled continous outcomes such as sales or prices with independent variables. Logistic regression is used to model the probability of a categorical outcome/dependent variable. This variable will have 2 options which are either a 1 or a 0, and the logistic regression model will attempt to predict P(Y=1) as a function of X.

**The Logistic Regression Assumptions**

* Binary logistic regression requires the dependent variable to be binary.

* For a binary regression, the factor level 1 of the dependent variable should represent the desired outcome.

* Only the meaningful variables should be included.
The independent variables should be independent of each other. That is, the model should have little or no multicollinearity. Multicollinearity occurs when a linear combination of one or more independent variables is collinear with another variable. This charactersitic has a habit of inflating the variance of the parameter estimate in question and can wrongly accept the null hypothesis that the variable is not significantly different from zero.

* The independent variables are linearly related to the log odds.
* Logistic regression requires quite large sample sizes.

Keeping the above assumptions in mind, let’s look at a dataset. The dataset is the success rate of people applying for a specific position. Each candidates Graduate Management Admission Test score, GPA score and work expreience were also recorded. The aim of this analyis is to determine which variables should be used when making a decision to accept or reject a candidate.

We will read the data into a dataframe called candidates and this is outlined below:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
from statsmodels.compat import lzip
import statsmodels.formula.api as sm
import statsmodels.stats.stattools as st
import statsmodels.stats.api as sms
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
plt.rc("font", size=14)
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

candidates = {'gmat': [780,750,690,710,680,730,690,720,740,690,610,690,710,680,770,610,580,650,540,590,620,600,550,550,570,670,660,580,650,660,640,620,660,660,680,650,670,580,590,690],
              'gpa': [4,3.9,3.3,3.7,3.9,3.7,2.3,3.3,3.3,1.7,2.7,3.7,3.7,3.3,3.3,3,2.7,3.7,2.7,2.3,3.3,2,2.3,2.7,3,3.3,3.7,2.3,3.7,3.3,3,2.7,4,3.3,3.3,2.3,2.7,3.3,1.7,3.7],
              'work_experience': [3,4,3,5,4,6,1,4,5,1,3,5,6,4,3,1,4,6,2,3,2,1,4,1,2,6,4,2,6,5,1,2,4,6,5,1,2,1,4,5],
              'admitted': [1,1,1,1,1,1,0,1,1,0,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,0,0,1,1,1,0,0,0,0,1]
              }
df = pd.DataFrame(candidates,columns= ['gmat', 'gpa','work_experience','admitted'])


df.describe()
#from google.colab import files
#df.to_csv('candidates.csv')
#files.download('candidates.csv')

We have now imported the data and printed a summary analysis of the data using the Pandas "describe" method. By doing this we can easily see that the problems is well balanced between accept and reject for the admitted variable. This is very convenient, as normally this would not be the case. When problems are imbalanced we should really deal with this as part of the problem.

The next thing you will have noticed is the mean response for the 3 independent varibles which are as follows:
>gmat |	gpa |	work_experience
>--- | --- | ---
>654.00	| 3.095	| 3.425

We can see that the gpa and work_experience variables are of a similar scale but gmat is a factor of 100 greater. Many people when starting out in their analytics journey feel the need to standardise their data as a preprocessing step. This is not necessary if you are dealing with a linear model, as we are not using complex optimisation routines. We will look at differing methods to standardise data in the next MOOC but **in the case of logistic or linear regression there is absolutely no need to standardise your independent variables**. In fact if you do this you will be actually reduce the volume of information from your model, thus reducing the models predictive power.

You may want to center your independent variables as this may improve the interpretation of the intercept. When you center a variable the intercept becomes the mean of the Y variable at the value you centered on, but importantly the slopes/paremeter estimates will not change. But for this logistic model we will find that model will fail as well get "perfect seperation". This is usually caused by over specification of the model (To many variables). Try it out and see what happens. See if you can get results using 2 variables and no intercept. What do you notice about the p-values.

In [ ]:
#df['gmat_t']=(df['gmat']-df['gmat'].min())/(df['gmat'].max()-df['gmat'].min())
#print(df['gmat_t'])
#df['gpa']=df['gpa']-df['gpa'].mean()
#df['work_experience']=df['work_experience']-df['work_experience'].mean()
#df['gmat']=df['gmat']-df['gmat'].mean()
sns.countplot(x='admitted',data=df,palette='hls')
plt.show()
plt.savefig('count_plot')

We see from the above histogram that the "admitted" variable is well balanced. Generally, we should really complete a number of plots and a correlation analysis to examine the relationship between variables. If 2 variables are highly correlated then it is appropriate to drop one of these variables from the model.

In [ ]:
sns.set()
%matplotlib inline
df.plot.scatter(x='gpa', y='work_experience')

In [ ]:
sns.set()
%matplotlib inline
df.plot.scatter(x='gpa', y='gmat')

In [ ]:
corr = df[['gpa','work_experience','gmat']].corr()
corr.style.background_gradient(cmap='coolwarm')

# The 3 independent variables do appear to have some correlation between them, but we cannot be sure how this will affect our model. In order to assess this we should really examine the multi-collinearity between all the independent variables. The code below shows how we can assess multi-collinearity using a term known as the "variance inflation factor" or VIF. VIF is calculated by regressing a independent variable against all the other variables and using the following formula:
$$ V.I.F. = 1 / (1 - R^2). $$

If your VIF factor is >10 then you really need to consider dropping variables from your model. In the example below we can see that the following equations give the VIF calculated from the code:

<br/>

$gmat=\alpha{} + \beta{}_1.gpa +\beta{}_2.workexperience$ gives a VIF of 1.88

$gpa=\alpha{} + \beta{}_1.gmat +\beta{}_2.workexperience$ gives a VIF of 1.54

$workexperience=\alpha{} + \beta{}_1.gpa +\beta{}_2.gmat$ gives a VIF of 1.47

<br/>

While gmat and gpa only had a correlation coefficent of 0.56 which is not excessively high we see that the there is little multi-collinearity between the variables.


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from statsmodels.api import add_constant
X=df[['gpa','work_experience','gmat']]
X = add_constant(X)
y=df['admitted']
vif = [variance_inflation_factor(X.to_numpy(), i) for i in range(X.to_numpy().shape[1])]

print(vif[1:])


Now we will run a number of logistic regression models and see which ones make most sence.

In [ ]:
import statsmodels.api as sm

import statsmodels.discrete.discrete_model as sml
X=df[['gpa','gmat','work_experience']]
#X=df[['gpa','work_experience']]
#X = add_constant(X)
#X=df[['work_experience']]
logit = sml.Logit(y, X).fit()
print(logit.summary())

*   Pseudo R-squ. is similar to the $R^2$ in linear regression
*   LLR p-value: tells if the model is significantly better than a null model
* The confusion matrix tells which categories are more likely to be predicted correctly and incorectly.

Now by interchanging GPA, GMAT and the intercept we get differing values for the Rseudo $R^2$ or none at all as the model fails. In order to determine which model is most appropriate we would take the  Pseudo $R^2$, the pvalues and the LLR pvalue into account. We would then examine the Accuarcy, F1-score and the Area under the curve. Sometimes it is a tough call but in this situation we would probably use the gmat, gpa and work_experience variables with no intercept, as they give the  highest $R^2$ and LLT P-value is highly significant. However, the coeffficent of the GMAT variable is in the wrong direction. You will notice that it is negative. This tells us that it has a slight negative impact on the odds of being admitted. You will also notice if you leave the GMAT variable out that the coeficent for GPA swaps diection.



Another, issue is we do not have a intercept. You will see that if you swap the intercept with the gmat variable we get a Pseudo $R^2$ of 1. Also when we examine the Accuracy of this model we get a perfect score. One needs to be very careful when picking a model like this as you cannot be sure that the this model would scale very well. A real point to be worried about with this model is the size of the standard errors for all the variables including the intercept.

You could argue that if you have a zero GPA, GMAT and work experience then you would expect the odds of being addmitted being zero.

If you add a constant into the model you will find that it fails. This usually occurs because the model is overspecified or unstable, and can mean that a number of variables/predictors are highly correlated with the intercept, or you have to many variables in the model or the dataset is to small.

Also, most importantly you will notice the standard error inflates dramitically when we include an intercept, thus we cannot definitively tell which variables are useful. Taking all of this evidence into account would suggest we ignore the intercept term.

We will now examine the accuracy, F1 score and the AUC.




In [ ]:
confusion_matrix = pd.crosstab(y, np.round_(logit.predict(),0), rownames=['Actual'], colnames=['Predicted'])
sns.heatmap(confusion_matrix, annot=True)

In the following code window we can run a clasification report which gives a number of statistics. The explaination for these statistics is as follows:

* accuracy (correct decisions/total no. of decisions)
* precision (True positives/(True Positives+False Positivies))
* recall (True positives/(True Positives + False Negatives))
* f1-scrore (Formula including precision and recall)
* support (The number of items in the class)

In classification problems we need to be very careful not to over rely on the accuracy score. It can often be the case that a high accuracy could be found, however, we may need to put more emphasis on identifying a particular class. So for example if we were trying to identify critically ill patients from a rare disease we would most likely get a high accuaracy as we would identify the majority of patients who were not ill, but diagnose someone as not being ill when in fact they are seriously ill. This is the reason why some authors report a F1-score. However, this to has been criticized has it treats precision and recall equally.  



In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y, np.round_(logit.predict(),0)))

Finally for logistic regression models we should build a Receiver Operating Characteristic (ROC) curve and determine the Area Under the Curve (AUC)
A ROC curve, is a graphical plot that illustrates the diagnostic ability of a binary classifier system as its discrimination threshold is varied. The ROC curve is created by plotting the true positive rate against the false positive rate at various threshold settings. The higher the AUC the better the classifier.

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

logit_roc_auc = roc_auc_score(y, logit.predict(X))
fpr, tpr, thresholds = roc_curve(y, logit.predict(X))
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

So we have only really shown how to run the logistic model, but we have not really examined the formula for it or the interpretation of the parameter estimates. We will address these issues in MOOC5 when we introduce the theory behind Generalised Linear Model.

Finally, you will all want to try to predict class or continous variables. If you are doing this run the models on a "training" set. This is a subset of your data that your models will be trained on. The remaining part of the data will be treated as a test set. When your model is finalised on the training set you will then test it on your test set. There are a number of ways to select training and test sets and you always have to make sure they are properly balanced. You will learn a lot more about this in Machine Learning.